# Ex. 07 - Simulazione ensemble NVT con algoritmo Metropolis

## Introduzione
In questo esercizio sono state studiate - mediante l'uso di algoritmi MC - le tre fasi termodinamiche (solida, liquida, gassosa) di un fluido Lennard-Jones sottoposto ai vincoli che definiscono l'ensemble canonico: numero di particelle ($N=108$), volume ($V$) e temperatura ($T$) costanti nel tempo. La distribuzione di probabilità $p(\mu)$ che descrive gli stati $\mu=\{\vec x_i,\vec v_i\}$ di questo sistema all'equilibrio è quindi quella di Boltzmann:

$$ p(\mu) \propto e^{-H(\mu)/T}$$

dove $H(\mu)=U(\{\vec x_i\})+K(\{\vec v_i\})$ è l'hamiltoniana del sistema, $K(\{\vec v_i\})$ l'energia cinetica delle particelle e $U(\{\vec x_i\})=\sum_{i<j}^N \nu_{L-J}(|\vec x_i-\vec x_j|)$ l'energia potenziale ottenuta dalla somma dei potenziali a due corpi di Lennard-Jones:

$$\nu_{L-J}(|\vec x_i-\vec x_j|)=4\epsilon\big[\big(\frac{\sigma}{|\vec x_i-\vec x_j|}\big)^{12}-\big(\frac{\sigma}{|\vec x_i-\vec x_j|}\big)^6\big]$$

Per riprodurre questo sistema, la parte configurazionale della distribuzione di Boltzmann ($\propto e^{-V(\{\vec x_i\})/T}$) è stata campionata con l'algoritmo Metropolis. Terminata la fase di equilibrazione, sono stati calcolati - con il metodo del blocking - i valori di aspettazione con incertezza statistica delle seguenti grandezze:

- energia potenziale per particella $U/N$
- pressione $P=\rho T + \frac{\epsilon}{3V} \sum_{i<j1}^{N}48 \left[ \left(\frac{\sigma}{|\vec x_i -\vec x_j|}\right)^{12} - 
\frac{1}{2} \left(\frac{\sigma}{|\vec{x}_i -\vec{x}_j|}\right)^6 \right]$
- funzione di distribuzione radiale $g(r)=\frac{1}{\rho N \Delta V(r)}\sum_{i<j}^N \delta(r-|\vec x_i-\vec x_j|)$ , con $\rho$ è la densità numero e $\Delta V(r)=\frac{4\pi}{3}[(r+dr)^3-r^3]$. La funzione di distribuzione radiale è stata introdotta nell'esercizio 4 di dinamica molecolare e quindi qui saranno riportati solo i risultati, per cui valgono le stesse osservazioni.

Le simulazioni sono state condotte nelle condizioni termodinamiche seguenti:
- fase solida: $\rho^\star = 1.1$, $T^\star = 0.8$ ($r_c = 2.2)$
- fase liquida: $\rho^\star = 0.8$, $T^\star = 1.1$, ($r_c = 2.5)$
- fase gassosa: $\rho^\star = 0.05$, $T^\star = 1.2$, ($r_c = 5.0)$

in ciascuna delle quali sono stati regolati i parametri:
- dimensione del passo $\delta$ per ciascuna coordinata al fine di ottenere un'accettazione del Metropolis pari a circa il $50\%$
- tempo di equilibrazione $\tau_{eq}$ (in passi MC)
- dimensione $L$ dei blocchi minima necessaria a far decadere le correlazioni per le grandezze calcolate 

Come nell'esercizio di dinamica molecolare, anche in questo caso nel corso della simulazione tutte le grandezze sono state espresse in unità delle scale di lunghezza $\sigma$, di energia $\epsilon$ e di massa $m$ caratteristiche del sistema in esame e solo in seguito sono state riscalate. In particolare, sono stati presentati i risultati per l'Argon e il Krypton, entrambi ben descritti da un potenziale di tipo L-J e caratterizzati dai seguenti valori di $\sigma,\epsilon,m$:
- Argon $\sigma = 0.34$ nm, $\epsilon/k_B = 120$ K, $m=39.948$ amu
- Krypton: $\sigma = 0.364$ nm, $\epsilon/k_B = 164$ K, $m=83.798$ amu



## Passo Metropolis
Per generare nuove configurazioni del sistema sono stati effettuati, per ciascuna coordinata spaziale, passi distribuiti in maniera uniforme nell'intervallo $[-\delta,\delta]$. Il parametro $\delta$ è stato scelto per ciascuna fase termodinamica al fine di ottenere un rate di accettazione pari a circa il $50\%$:

- solido: $\delta=0.06$
- liquido: $\delta=0.1$
- gas: $\delta=3.0$

NB: nel file input.dat il parametro da inserire è $2\delta$

Non è stato possibile ottenere un rate di accettazione inferiore al 60% per il gas. Quest'ultimo, infatti, è caratterizzato da una densità molto più bassa rispetto al liquido e al solido, il che implica grandi distanze tra particelle. A grandi distanze il potenziale è $\textit{debolmente}$ attrattivo ed è quindi necessario effettuare passi piuttosto grandi per ottenere configurazioni ad energie apprezzabilmente diverse. Inoltre, la temperatura è più alta rispetto al caso di solido e liquido, con la conseguenza di rendere ancora più vicina a $1$ la probabilità di transizione $e^{\Delta V/T}$. Le condizioni periodiche al contorno rendono però impossibile aumentare $\delta$ a piacimento per ottenere l'accettazione desiderata. Si è concluso quidi che un valore di $\delta$ pari a 3 potesse essere un buon compresso tra rate di accettazione e limiti della simulazione dovuti alla taglia del box($L_{box}\simeq13$).

## Equilibrazione
Prima di calcolare le grandezze fisiche di interesse, sono stati effettuati alcuni passi MC fino a quando il sistema ha raggiunto l'equilibrio termodinamico. Il tempo di equilibrazione $\tau_{eq}$ è stato determinato per via empirica plottando i valori istantanei dell'energia potenziale e della pressione in funzione del numero di passi MC e attendendo che questi fluttuassero attorno al loro valor medio all'equilibrio. Sono stati adottati tempi di equilibrazione diversi a seconda della fase termodinamica simulata:

- solido: $\tau_{eq}=100$ passi MC
- liquido: $\tau_{eq}=200$ passi MC
- gas: il sistema raggiunge l'equilibrio in pochi passi MC


## Stima dell'incertezza statistica
Uno dei problemi legati al Metropolis risiede nel fatto che una nuova configurazione viene generata a partire dalla configurazione al passo precedente, con la conseguenza di ottenere grandezze, funzioni delle configurazioni del sistema, autocorrelate positivamente. Per ottenere una stima corretta dell'incertezza statistica mediante il metodo del blocking è necessario però assicurarsi che le medie calcolate in ciascun blocco siano scorrelate tra loro, condizione necessaria per l'applicazione del teorema del limite centrale. Per fare ciò, sono stati adottati due metodi, che hanno richiesto la simulazione di $M=10^5$ passi MC e il calcolo dei valori istantanei di energia e pressione:

- calcolo della funzione di autocorrelazione di energia potenziale e pressione in funzione del numero di passi MC
- calcolo dell'incertezza statistica sui valori medi di energia potenziale e pressione in funzione del numero di passi $L$ di un blocco, con $L\in[10,5\cdot10^3]$

### Funzione di autocorrelazione e tempo di correlazione
La funzione di autocorrelazione temporale di una grandezza $O(t)$ è definita nel seguente modo:

$$Ac_{[O]}(t)=\frac{\big\langle O(t')O(t'+t)\big\rangle_{t'}-\big\langle O\big\rangle^2}{\sigma_O^2}$$

Per come è stata definita, $Ac_{[O]}(0)=1$ (massima autocorrelazione) mentre $Ac_{[O]}(t)=0$ indica totale scorrelazione tra valori di $O$ misurati a distanza di tempo t. 
Quello che ci aspetta nel caso del sistema in esame (sistema 3D di particelle con interazione L-J in ensemble canonico all'equilibrio termodinamico) è che la funzione di autocorrelazione decada a zero con un andamento approssimabile ad un esponenziale decrescente $e^{-t/\tau_c}$, dove $\tau_c$ viene definito il tempo di correlazione e identifica la scala di tempo su cui le correlazioni decadono. Il tempo effettivo a cui le correlazioni sono decadute a zero può però essere pari a qualche volta $\tau_c$. 

Nel caso di simulazioni MC il "tempo" $t$ è discreto ed occorre trovare un'espressione di $Ac_{[O]}(t)$ utile alla sua implementazione. A partire da un set di campionamenti di una grandezza $O$ ottenuto da una simulazioni di $t_{max}$ passi MC, la funzione di autocorrelazione può essere calcolata nel modo seguente:

$$Ac_{[O]}(t)=\frac{\frac{1}{t_{max}-t}\sum_{t'=0}^{t_{max}-t}O(t')O(t'+t)-\frac{1}{t_{max}-t}\sum_{t'=0}^{t_{max}-t}O(t')\times\frac{1}{t_{max}-t}\sum_{t'=0}^{t_{max}-t}O(t'+t)}{\frac{1}{t_{max}}\sum_{t'=0}^{t_{max}}O^2(t')-\big(\frac{1}{t_{max}}\sum_{t'=0}{t_{max}}O(t')\big)^2}$$

Per il calcolo della funzione di autocorrelazione ad un tempo di t, il numero di termini disponibili per le medie al numeratore è pari a $t_{max}-t$: questo significa che all'aumentare di $t$ le medie vengono effettuate su un campione sempre più piccolo, con conseguente aumento delle fluttuazioni di $Ac_{[O]}(t)$. È necessario quindi scegliere un range temporale per $t$ che sia molto più piccolo di $t_{max}$ ma che allo stesso tempo permetta di osservare il decadimento atteso.

In questo esercizio $t_{max}=10^5$, mentre il range di $t$ è stato scelto in base alla fase termodinamica simulata.

Mediante un fit con esponenziale decrescente della curva ottenuta dalla simulazione è stato quindi ricavato il tempo di correlazione $\tau_c$ relativo alle grandezze energia interna e pressione. A seconda della fase termodinamica, il tempo impiegato dalla funzione di autocorrelazione per raggiungere lo $0$ è all'incirca:

- solido: $100-120$ passi MC
- liquido: $300-400$ passi MC
- gas: $50-70$ passi MC

La taglia dei blocchi dovrà quindi essere uguale o maggiore al tempo di decadimento delle correlazioni.

### Data blocking e dimensione dei blocchi
Un altro metodo per ricavare la dimensione dei blocchi $L$ minima che permette di ottenere - per una grandezza $O$ - valori medi sul blocco $\langle O\rangle_{L}$ scorrelati tra loro consiste nello stimare l'errore statistico $\sigma_O$ con il metodo del blocking per diversi valori di $L$ (mantenendo costante il numero totale $M$ di passi MC) e osservarne l'andamento.



Quello che ci si aspetta è che per valori di $L$ più piccoli del tempo di decadimento delle correlazioni, all'aumentare di $L$ l'errore stimato aumenti fino ad approcciare il vero errore statistico. Una volta raggiunto questo valore limite, incrementare $L$ non dovrebbe causare una variazione dell'errore dato che i blocchi sono sicuramente scorrelati e il teorema del limite centrale è applicabile. Nella pratica è comunque opportuno non incrementare eccessivamente $L$ per evitare di ottenere un numero di blocchi $N=M/L$ troppo scarso su cui effettuare il blocking.

A seconda della fase termodinamica, il valore di $L$ a cui l'errore statistico stimato raggiunge - sia per l'energia interna che per la pressione - il valore limite (quello "vero") è:

- solido: $150$ passi MC
- liquido: $800$ passi MC
- gas: $80-100$ passi MC


Sulla base dei risultati trovati con i metodi illustrati, sono state quindi condotte simulazioni di $M=10^5$ passi MC utilizzando:

- solido: $N=500$ blocchi da $L=200$ passi
- liquido: $N=125$ blocchi da $L=800$ passi
- gas: $N=1000$ blocchi da $L=100$ passi


Tutti i grafici e i risultati sono stati riportati, in base alla fase termodinamica, nei jupyter "Solid.ipynb", "Liquid.ipynb" e "Gas.ipynb".